# Note at the beginning


It seems like there might be a mistake here. Both the booking_details and reviews contain practically the same data. The booking_details Excel file actually has only one sheet inside, called "reviews," which could indicate an honest mistake.

# Load modules

In [26]:
from pandasql import sqldf
import pandas as pd
import re

# Import data

In [27]:
# load data
booking_details = pd.read_excel('data/booking details-Boston.xlsx')
listings = pd.read_excel('data/listings-Boston.xlsx')
reviews = pd.read_excel('data/reviews-Boston.xlsx')

# Task 1: Rentals with the most availability

In [35]:
qry = '''
SELECT 
    id as rental_id, 
    name reantal_name, 
    availability_365 as availability

FROM 
    listings

ORDER BY 
    availability DESC
'''

sqldf(qry)

,rental_id,reantal_name,availability
0,10813,Rental unit in Boston ¬∑ ‚òÖ5.0 ¬∑ Studio ¬∑ 1...,365
1,10986,Condo in Boston ¬∑ Studio ¬∑ 1 bed ¬∑ 1 bath,365
2,973695,Condo in Boston ¬∑ ‚òÖ4.76 ¬∑ 1 bedroom ¬∑ 2 b...,365
3,1340649,Condo in Boston ¬∑ ‚òÖ4.81 ¬∑ 1 bedroom ¬∑ 1 b...,365
4,2014651,Rental unit in Boston ¬∑ ‚òÖ4.62 ¬∑ 1 bedroom ...,365
...,...,...,...
3968,837439628788914048,Rental unit in Boston ¬∑ 1 bedroom ¬∑ 1 bed ¬∑...,0
3969,846239063484503040,Rental unit in Boston ¬∑ Studio ¬∑ 1 bed ¬∑ 1 ...,0
3970,853391813264547968,Rental unit in Boston ¬∑ 1 bedroom ¬∑ 1 bed ¬∑...,0
3971,881255998576454016,Home in Boston ¬∑ ‚òÖNew ¬∑ 4 bedrooms ¬∑ 4 be...,0


# Task 2: Total rental revenue for private room rentals in each Boston neighborhood

In [60]:
qry = '''
SELECT 
    l.neighbourhood,
    SUM(bd.revenue) total_revenue
    
FROM
    booking_details as bd
    
    left join listings as l
        on bd.listing_id = l.id
        
WHERE
    l.room_type == "Private room"

GROUP BY 
    1

ORDER BY 
    2 DESC
'''
sqldf(qry)

,neighbourhood,total_revenue
0,East Boston,30674014
1,Dorchester,27855573
2,Roxbury,16448552
3,Jamaica Plain,9850777
4,South Boston,8008997
5,Brighton,7502684
6,Allston,7378986
7,South End,5807633
8,Downtown,4825582
9,Fenway,4364733


# Task 3: Total rental revenue for private room rentals in each Boston neighborhood that have at least $200K in bookings

In [61]:
qry = '''
SELECT 
    l.neighbourhood,
    SUM(bd.revenue) total_revenue
    
FROM
    booking_details as bd
    
    left join listings as l
        on bd.listing_id = l.id
        
WHERE
    l.room_type == "Private room"

GROUP BY 
    1
    
HAVING 
    total_revenue > 200000

ORDER BY 
    2 DESC
'''
sqldf(qry)

,neighbourhood,total_revenue
0,East Boston,30674014
1,Dorchester,27855573
2,Roxbury,16448552
3,Jamaica Plain,9850777
4,South Boston,8008997
5,Brighton,7502684
6,Allston,7378986
7,South End,5807633
8,Downtown,4825582
9,Fenway,4364733


# Task 4: Top 5 rentals that have the most booking revenue in each room type category

In [80]:
qry = '''
WITH total_revenue as (
    SELECT 
        l.room_type,
        l.name as rental_name,
        SUM(bd.revenue) total_revenue

    FROM 
        booking_details as bd
        
        left join listings as l
            on bd.listing_id = l.id
        
    GROUP BY 
        1, 2

),
rank as (
    SELECT 
        tr.*,
        ROW_NUMBER() OVER(PARTITION BY tr.room_type ORDER BY tr.total_revenue DESC) as rank
        
    FROM 
        total_revenue as tr    

)
SELECT 
    * 

FROM 
    rank

WHERE 
    rank <= 5

ORDER BY 
    1, 3 DESC
'''
sqldf(qry)

,room_type,rental_name,total_revenue,rank
0,Entire home/apt,Rental unit in Boston ¬∑ ‚òÖ4.89 ¬∑ Studio ¬∑ ...,2808514,1
1,Entire home/apt,Rental unit in Boston ¬∑ ‚òÖ4.76 ¬∑ Studio ¬∑ ...,2552690,2
2,Entire home/apt,Rental unit in Boston ¬∑ ‚òÖ4.65 ¬∑ Studio ¬∑ ...,2481765,3
3,Entire home/apt,Rental unit in Boston ¬∑ ‚òÖ4.92 ¬∑ 2 bedrooms...,2448739,4
4,Entire home/apt,Rental unit in Boston ¬∑ ‚òÖ4.62 ¬∑ Studio ¬∑ ...,2174764,5
5,Hotel room,Hotel in BOSTON ¬∑ ‚òÖ4.67 ¬∑ 1 bedroom ¬∑ 1 b...,857737,1
6,Hotel room,Hotel in Boston ¬∑ ‚òÖ4.38 ¬∑ 1 bedroom ¬∑ 1 b...,537237,2
7,Hotel room,Hotel in BOSTON ¬∑ ‚òÖ4.71 ¬∑ 1 bedroom ¬∑ 2 b...,533785,3
8,Hotel room,Boutique hotel in Boston ¬∑ ‚òÖ4.77 ¬∑ 1 bedro...,230234,4
9,Hotel room,Boutique hotel in Boston ¬∑ ‚òÖ4.78 ¬∑ 1 bedro...,228625,5


# Task 5: Partition the listing name to be able to filter by the number of bedrooms and bathrooms

In [136]:
# Same regex expression can be used to extract number of bedrooms and bathrooms from the name column in the sql
def get_number_before_string(string, pattern):
    match = re.search(r'(\d+)\s' + pattern, string)
    number = match.group(1) if match else 0
    return number


new_filters = ['bedroom', 'bed', 'bath']

for new_filter in new_filters:
    listings[new_filter] = listings['name'].apply(lambda x: get_number_before_string(x, new_filter))

listings.head()

# Task 6: Fireplaces feature flag analysis

## Overall Impact 

In [145]:
## How has the addition of the fireplace feature flag affected booking rates? 
qry = '''
SELECT 
    CASE
        WHEN date(date) <= "2018-01-01" THEN 0
        ELSE 1
    END as fireplace_feature,
    count(*) as total_bookings
    
FROM 
    booking_details as bd
    
GROUP BY 
    1
    
'''
sqldf(qry)

,fireplace_feature,total_bookings
0,0,29619
1,1,144893


In [148]:
## Has there been any change in the average booking value since the fireplace feature flag was introduced?
qry = '''
SELECT 
    CASE
        WHEN date(date) <= "2018-01-01" THEN 0
        ELSE 1
    END as fireplace_feature,
    ROUND(AVG(revenue), 2) as average_booking_value
    
FROM 
    booking_details as bd
    
GROUP BY 
    1
'''
sqldf(qry)

,fireplace_feature,average_booking_value
0,0,2545.31
1,1,2552.22


## Host Adoption Rate:

In [170]:
## What percentage of hosts have adopted the fireplace feature flag since its introduction?
qry = '''
WITH src as (
    SELECT DISTINCT 
        bd.listing_id,
        l.fireplace_feature_flag
        
    FROM 
        booking_details as bd
        
        left join listings as l
            on bd.listing_id = l.id
        
    WHERE 
        date(date) > "2018-01-01"
),
totals as (
    SELECT 
        SUM(
            CASE    
                WHEN fireplace_feature_flag != "None" THEN 1
                ELSE 0
            END
        ) as fireplace_feature_flag,
        COUNT(*) as total_rentals_after_flag_change  
    FROM 
        src  
)
SELECT 
    ROUND(CAST(fireplace_feature_flag AS FLOAT) / total_rentals_after_flag_change, 2) as adoption_rate
FROM
    totals


'''
sqldf(qry)

,adoption_rate
0,0.91


In [ ]:
## Are there any trends in host adoption rates of the fireplace feature flag?

# This is unfortunaelty impossible to answer with the data provided. 
# There is no possibility to track when exactly the feature was introduced as data seams to be historically overwriten.

## Guest Preferences:

In [173]:
# Are guests more likely to book rentals with the fireplace feature flag
qry = '''
SELECT  
    CASE    
        WHEN fireplace_feature_flag != "None" THEN 1
        ELSE 0
    END fireplace_present,
    count(*) as total_bookings
    
FROM 
    booking_details as bd
    
    left join listings as l
        on bd.listing_id = l.id
    
WHERE 
    date(date) > "2018-01-01"
    
GROUP BY 
    1


'''
sqldf(qry)

,fireplace_present,total_bookings
0,0,11661
1,1,133232


## User Feedback Analysis

In [199]:
# What is guests satisfaction about rentals with fireplaces in their reviews?
qry = '''
WITH src as (
        SELECT  
        CASE    
            WHEN fireplace_feature_flag != "None" THEN 1
            ELSE 0
        END fireplace_present,
        bd.review_stars,
        count(*) as total_reviews
        
    FROM 
        booking_details as bd
        
        left join listings as l
            on bd.listing_id = l.id
        
    WHERE 
        date(date) > "2018-01-01"
        
    GROUP BY 
        1, 2
        
    ORDER BY 
        1, 2
)
SELECT 
    s.*,
    ROUND(
        CAST(total_reviews AS FLOAT) / 
            CAST(SUM(total_reviews) OVER(PARTITION BY s.fireplace_present) AS FLOAT),
        2
    )as distribution

FROM 
    src as s

'''
sqldf(qry)

,fireplace_present,review_stars,total_reviews,distribution
0,0,NaN,1,0.00
1,0,1.0,2248,0.19
2,0,2.0,2390,0.20
3,0,3.0,2368,0.20
4,0,4.0,2310,0.20
5,0,5.0,2344,0.20
6,1,NaN,12,0.00
7,1,1.0,26789,0.20
8,1,2.0,26702,0.20
9,1,3.0,26626,0.20
